In [6]:
import tensorflow as tf
import numpy as np
import tensorflow.keras as keras
import tensorflow.keras.backend as K
import matplotlib.pyplot as plt
import time

import sys
sys.path.append("../../")
from Seismic_wave_inversion_PINN.tf_model_utils import *
from Seismic_wave_inversion_PINN.data_utils import *

plt.rcParams.update(plt.rcParamsDefault)
tf.keras.backend.set_floatx("float32")

%load_ext autoreload
%autoreload 2

print(tf.config.list_physical_devices('GPU'))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


In [7]:
model = keras.models.Sequential([
	keras.layers.Dense(128, input_shape = (1,), activation = "tanh", kernel_initializer = "random_normal"),
	keras.layers.Dense(128, activation = "tanh", kernel_initializer = "random_normal"),
	keras.layers.Dense(128, activation = "tanh", kernel_initializer = "random_normal"),
	keras.layers.Dense(128, activation = "tanh", kernel_initializer = "random_normal"),
	keras.layers.Dense(1, use_bias = False)
])

loss_func = keras.losses.MeanSquaredError()

In [8]:
def get_residue_loss(tape, x, y):
	dy_dx = tape.gradient(y, x)
	return loss_func(dy_dx, 1.0)

def step(x):
	with tf.GradientTape(persistent = True) as tape:
		tape.watch(x)
		y_pred = model(x)
		loss = get_residue_loss(tape, x, y_pred)
	grads = tape.gradient(loss, model.trainable_variables)
	optimizer.apply_gradients(zip(grads, model.trainable_variables))
	del tape

In [9]:
x = tf.constant(np.random.random((1000, 1)), dtype = tf.float32)

In [10]:
optimizer = keras.optimizers.SGD(lr = 0.01)
epochs = 5000

import time
start_time = time.time()
for i in range(1, epochs+1):
	step(x)
print("AVG epoch time: ", (time.time() - start_time)/epochs)

AVG epoch time:  0.01430511293411255
